In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
import sys
sys.path.append('/app')

In [4]:
from vexa import VexaAPI, VexaAuth
vexa = VexaAPI()
await vexa.get_user_info()

Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


{'id': 'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
 'email': '2280905@gmail.com',
 'is_allowed_send_init_message': True,
 'username': 'Dmitriy Grankin',
 'first_name': 'Dmitriy',
 'last_name': 'Grankin',
 'image': None,
 'created_timestamp': '2024-05-28T11:16:50.894711Z',
 'updated_timestamp': '2024-11-21T22:19:24.910083Z'}

In [5]:
vexaauth = VexaAuth()


In [7]:
await vexaauth.get_user_token(email='a.zakharov@citydrive.ru')

'ffc47fdc1f79475994fb93a29a394a09'

In [4]:
import sys
sys.path.append('/app')


In [5]:
from chat import MeetingChatManager
from psql_helpers import get_session
from uuid import UUID


In [6]:
from IPython.display import clear_output,display,Markdown


In [7]:
from IPython.display import clear_output, display, Markdown
from functools import wraps
from typing import AsyncIterator

def jupyter_stream_output(markdown: bool = True):
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            chunks = []
            async for response in func(*args, **kwargs):
                if isinstance(response, str):
                    chunks.append(response)
                    clear_output(wait=True)
                    if markdown:
                        display(Markdown(''.join(chunks)))
                    else:
                        print(''.join(chunks), flush=True)
                else:
                    return response
            return ''.join(chunks)
        return wrapper
    return decorator

In [8]:
from chat import MeetingSummaryContextProvider

In [37]:
from typing import List, Dict
import requests
BASE_URL = "http://localhost:8765"
headers = {"Authorization": f"Bearer {vexa.token}", "Content-Type": "application/json"}

In [38]:
def get_meetings_by_speakers(speakers: List[str], limit: int = 50, offset: int = 0) -> Dict:
    return requests.post(
        f"{BASE_URL}/meetings/by-speakers",
        headers=headers,
        json={"speakers": speakers, "limit": limit, "offset": offset}
    ).json()

In [39]:
meeting_ids = get_meetings_by_speakers(['Artem Puzik'])

In [41]:
meeting_ids = [m['meeting_id'] for m in meeting_ids['meetings']]


In [15]:
user_id = vexa.user_id

In [52]:
thread_id = None

In [57]:

@jupyter_stream_output(markdown=True)
async def chat_with_meeting(user_id: str, meeting_ids: list[UUID], query: str,thread_id: str):
    async with get_session() as session:
        summary_provider = MeetingSummaryContextProvider(meeting_ids, include_discussion_points=True)
        chat_manager = MeetingChatManager(session, context_provider=summary_provider)
        async for response in chat_manager.chat(
            user_id=user_id,
            query=query,
            meeting_ids=meeting_ids,
            thread_id=thread_id,
            temperature=0.7
        ):
            yield response

# Usage
response = await chat_with_meeting(
    thread_id=thread_id,
    user_id='ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
    meeting_ids=meeting_ids,
    query="in english"
)

His first name in English is Dmitry.

In [56]:
thread_id = response['thread_id']

In [35]:
summary_provider = MeetingSummaryContextProvider(meeting_ids, include_discussion_points=True)
async with get_session() as session:
    chat_manager = MeetingChatManager(session, context_provider=summary_provider)
    async for response in chat_manager.chat(
        user_id=user_id,
        thread_id=thread_id,
        query=query,
        meeting_ids=meeting_ids,
        temperature=0.7
    ):
        print(response)


19:13:38 - vexa_api - INFO - Initializing ChatManager


thread_id='c81013c1-7c1c-4a1e-a26f-dd4b1890ec1f' user_id='ef7c085b-fdb5-4c94-b7b6-a61a3d04c210' thread_name='what is it baout?' messages=[Msg(role='user', content='what is it baout?', stage=None, service_content=None), Msg(role='assistant', content="The meetings discussed involve team collaboration and updates on technical processes within a software development context. \n\n1. **Meeting on Artem's Work and Pair Programming**: \n   - Dmitry Grankin suggested that Artem work on improving the interface under his guidance to enhance product and process understanding. \n   - Sergey Ryabenko expressed concerns about the scalability of the proposed team collaboration approach. \n   - Dmitry proposed an experiment with pair programming to assess its effectiveness and discussed the pros and cons of a no-code development approach. \n   - There was a broader discussion on the efficiency of teamwork, with Sergey noting that speed doesn't always equate to effectiveness.\n\n2. **Meeting on Authoriz

In [36]:
chat_manager.messages

[Msg(role='user', content='what is it baout?', stage=None, service_content=None),
 Msg(role='assistant', content="The meetings discussed involve team collaboration and updates on technical processes within a software development context. \n\n1. **Meeting on Artem's Work and Pair Programming**: \n   - Dmitry Grankin suggested that Artem work on improving the interface under his guidance to enhance product and process understanding. \n   - Sergey Ryabenko expressed concerns about the scalability of the proposed team collaboration approach. \n   - Dmitry proposed an experiment with pair programming to assess its effectiveness and discussed the pros and cons of a no-code development approach. \n   - There was a broader discussion on the efficiency of teamwork, with Sergey noting that speed doesn't always equate to effectiveness.\n\n2. **Meeting on Authorization Method Update and Testing**: \n   - Dmitry updated the authorization method to simplify the process and avoid multiple registratio

In [33]:
thread_id = response['thread_id']

In [23]:
response['thread_id']


'13909c3e-47f9-4999-a3b9-5534294dfbd1'